In [23]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import sys
import subprocess
import pdb
import time
import datetime
import math
import random
import _pickle as cPickle
from collections import defaultdict

from six.moves import zip_longest
import numpy as np

import tensorflow as tf
from tensorflow import distributions as tfd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk import word_tokenize

from data_structure import get_batches
from components import tf_log, sample_latents, compute_kl_loss, dynamic_rnn, dynamic_bi_rnn
from topic_model import TopicModel

from topic_beam_search_decoder import BeamSearchDecoder

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
PAD = '<pad>' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNK = '<unk>' # This has a vocab id, which is used to represent out-of-vocabulary words
BOS = '<p>' # This has a vocab id, which is used at the beginning of every decoder input sequence
EOS = '</p>' # This has a vocab id, which is used at the end of untruncated target sequences

# load data & set config

In [3]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

flags = tf.app.flags

flags.DEFINE_string('gpu', '1', 'visible gpu')

flags.DEFINE_string('mode', 'train', 'set train or eval')

flags.DEFINE_string('data_path', 'data/apnews/instances.pkl', 'path of data')
flags.DEFINE_string('modeldir', 'model/rnn_vae', 'directory of model')
flags.DEFINE_string('modelname', 'apnews', 'name of model')

flags.DEFINE_integer('epochs', 1000, 'epochs')
flags.DEFINE_integer('batch_size', 64, 'number of sentences in each batch')
flags.DEFINE_integer('log_period', 500, 'valid period')

flags.DEFINE_string('opt', 'Adagrad', 'optimizer')
flags.DEFINE_float('lr', 0.1, 'lr')
flags.DEFINE_float('reg', 0.1, 'regularization term')
flags.DEFINE_float('beta', 0.001, 'initial value of beta')
flags.DEFINE_float('grad_clip', 5., 'grad_clip')

flags.DEFINE_float('keep_prob', 0.8, 'dropout rate')
flags.DEFINE_float('word_keep_prob', 0.75, 'word dropout rate')

flags.DEFINE_integer('warmup', 5000, 'warmup period for KL')
flags.DEFINE_integer('warmup_topic', 0, 'warmup period for KL of topic')

flags.DEFINE_integer('beam_width', 2, 'beam_width')
flags.DEFINE_float('length_penalty_weight', 0.0, 'length_penalty_weight')

flags.DEFINE_integer('n_topic', 10, 'number of topic')
flags.DEFINE_integer('dim_hidden_bow', 256, 'dim of hidden bow')
flags.DEFINE_integer('dim_latent_topic', 32, 'dim of latent topic')
flags.DEFINE_integer('dim_emb', 256, 'dim_emb')
flags.DEFINE_integer('dim_hidden', 512, 'dim_hidden')
flags.DEFINE_integer('dim_latent', 32, 'dim_latent')


# for evaluation
flags.DEFINE_string('refdir', 'ref', 'refdir')
flags.DEFINE_string('outdir', 'out', 'outdir')

flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_bool('logtostderr', True, 'kernel')
flags.DEFINE_bool('showprefixforinfo', False, '')
flags.DEFINE_bool('verbosity', False, '')
# flags.DEFINE_integer('stderrthreshold', 20, 'kernel')

config = flags.FLAGS

flags.DEFINE_string('modelpath', os.path.join(config.modeldir, config.modelname), 'path of model')

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.data_path,'rb'))

In [6]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)

In [7]:
flags.DEFINE_integer('PAD_IDX', word_to_idx[PAD], 'PAD_IDX')
flags.DEFINE_integer('UNK_IDX', word_to_idx[UNK], 'UNK_IDX')
flags.DEFINE_integer('BOS_IDX', word_to_idx[BOS], 'BOS_IDX')
flags.DEFINE_integer('EOS_IDX', word_to_idx[EOS], 'EOS_IDX')

flags.DEFINE_integer('n_vocab', len(word_to_idx), 'n_vocab')
flags.DEFINE_integer('dim_bow', len(bow_idxs), 'dim_bow')

maximum_iterations = max([max([instance.max_sent_l for instance in batch]) for ct, batch in dev_batches])
flags.DEFINE_integer('maximum_iterations', maximum_iterations, 'maximum_iterations')    

# build language model 

## feed dict

In [9]:
tf.reset_default_graph()

t_variables = {}
t_variables['bow'] = tf.placeholder(tf.float32, [None, config.dim_bow])
t_variables['input_token_idxs'] = tf.placeholder(tf.int32, [None, None])
t_variables['dec_input_idxs'] = tf.placeholder(tf.int32, [None, None])
t_variables['dec_target_idxs'] = tf.placeholder(tf.int32, [None, None])
t_variables['doc_l'] = tf.placeholder(tf.int32, [None])
t_variables['sent_l'] = tf.placeholder(tf.int32, [None])
t_variables['keep_prob'] = tf.placeholder(tf.float32)

In [10]:
def get_feed_dict(batch, mode='train', assertion=False):
    def token_dropout(sent_idxs):
        sent_idxs_dropout = np.asarray(sent_idxs)
        sent_idxs_dropout[np.random.rand(len(sent_idxs)) > config.word_keep_prob] = config.UNK_IDX
        return list(sent_idxs_dropout)

    bow = np.array([instance.bow for instance in batch]).astype(np.float32)
    
    doc_l = np.array([len(instance.token_idxs) for instance in batch])
    
    feed_input_token_idxs_list = [sent_idxs for instance in batch for sent_idxs in instance.token_idxs]
    feed_dec_input_idxs_list = [[config.BOS_IDX] + token_dropout(sent_idxs) for sent_idxs in feed_input_token_idxs_list]
    feed_dec_target_idxs_list = [sent_idxs + [config.EOS_IDX]  for sent_idxs in feed_input_token_idxs_list]
        
    sent_l = np.array([len(sent_idxs) for sent_idxs in feed_input_token_idxs_list], np.int32)
    
    feed_input_token_idxs = pad_sequences(feed_input_token_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    feed_dec_input_idxs = pad_sequences(feed_dec_input_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    feed_dec_target_idxs = pad_sequences(feed_dec_target_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    
    if assertion:
        index = 0
        for instance in batch:
            for line_idxs in instance.token_idxs:
                assert feed_input_token_idxs_list[index] == line_idxs
                index += 1
        assert feed_input_token_idxs.shape[1] == np.max(sent_l)
        assert feed_dec_input_idxs.shape[1] == np.max(sent_l) + 1
        assert feed_dec_target_idxs.shape[1] == np.max(sent_l) + 1
    
    keep_prob = config.keep_prob if mode == 'train' else 1.0

    feed_dict = {
                t_variables['bow']: bow, 
                t_variables['doc_l']: doc_l, t_variables['sent_l']: sent_l, 
                t_variables['input_token_idxs']: feed_input_token_idxs, t_variables['dec_input_idxs']: feed_dec_input_idxs, t_variables['dec_target_idxs']: feed_dec_target_idxs, 
                t_variables['keep_prob']: keep_prob
    }
    return  feed_dict

In [11]:
def debug_shape(variables, sess_init=False):
#     if sess_init:
#         sess = tf.Session()
#         sess.run(tf.global_variables_initializer())
        
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)
            
    if sess_init: sess.close()

def debug_value(variables, return_value=False, sess_init=None):
#     if sess_init:
#         sess = tf.Session()
#         sess.run(tf.global_variables_initializer())

    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    if return_value: 
        return _variables
    else:
        for _variable, variable in zip(_variables, variables):
            if hasattr(variable, 'name'):
                print(variable.name, ':', _variable)
            else:
                print(_variable)
                
    if sess_init: sess.close()

## encoder

In [12]:
# input
input_token_idxs = t_variables['input_token_idxs']
batch_l, max_sent_l = tf.shape(input_token_idxs)[0], tf.shape(input_token_idxs)[1]
sent_l = t_variables['sent_l']

with tf.variable_scope('shared', reuse=False):
    embeddings = tf.get_variable('emb', [config.n_vocab, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of vocab

with tf.variable_scope('sent/enc', reuse=False):
    # get word embedding
    enc_input = tf.nn.embedding_lookup(embeddings, input_token_idxs)

    # get sentence embedding
    _, enc_state = dynamic_rnn(enc_input, sent_l, config.dim_hidden, t_variables['keep_prob'])

    means_infer = tf.keras.layers.Dense(units=config.dim_latent)(enc_state)
    logvars_infer = tf.keras.layers.Dense(units=config.dim_latent, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0))(enc_state)

    # latent vector from gaussian mixture    
    latents_input = sample_latents(means_infer, logvars_infer) 

## decoder

In [14]:
# prepare for decoding
dec_sent_l = tf.add(sent_l, 1)
dec_input_idxs = t_variables['dec_input_idxs']
dec_input = tf.nn.embedding_lookup(embeddings, dec_input_idxs)

dec_latents_input = tf.tile(tf.expand_dims(latents_input, 1), [1, tf.shape(dec_input)[1], 1])
dec_concat_input = tf.concat([dec_input, dec_latents_input], -1)

# decode for training
with tf.variable_scope('sent/dec/rnn', initializer=tf.contrib.layers.xavier_initializer(), dtype = tf.float32, reuse=tf.AUTO_REUSE):
    dec_cell = tf.contrib.rnn.GRUCell(config.dim_hidden)
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, output_keep_prob = t_variables['keep_prob'])

    dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu)(latents_input)
    
    helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_concat_input, sequence_length=dec_sent_l)

    train_decoder = tf.contrib.seq2seq.BasicDecoder(
        cell=dec_cell,
        helper=helper,
        initial_state=dec_initial_state)

    dec_outputs, _, output_sent_l = tf.contrib.seq2seq.dynamic_decode(train_decoder)
    
    output_layer = tf.layers.Dense(config.n_vocab, use_bias=False, name='out')
    output_logits = output_layer(dec_outputs.rnn_output)
    
    output_token_idxs = tf.argmax(output_logits, 2)

In [15]:
start_tokens = tf.fill([batch_l], config.BOS_IDX)
end_token = config.EOS_IDX

beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(dec_initial_state, multiplier=config.beam_width)
beam_latents_input = tf.contrib.seq2seq.tile_batch(latents_input, multiplier=config.beam_width) # added

beam_decoder = BeamSearchDecoder(
    cell=dec_cell,
    embedding=embeddings,
    start_tokens=start_tokens,
    end_token=end_token,
    initial_state=beam_dec_initial_state,
    beam_width=config.beam_width, 
    output_layer=output_layer,
    length_penalty_weight=config.length_penalty_weight,
    latents_input=beam_latents_input)

beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
    beam_decoder,
    maximum_iterations = config.maximum_iterations)

beam_output_token_idxs = beam_dec_outputs.predicted_ids[:, :, 0]

## language modeling cost

In [16]:
# target and mask
dec_target_idxs = t_variables['dec_target_idxs']
dec_mask_tokens = tf.sequence_mask(dec_sent_l, maxlen=max_sent_l+1, dtype=tf.float32)

# nll for each token (averaged over batch & sentence)
sent_loss_recon = tf.contrib.seq2seq.sequence_loss(output_logits, dec_target_idxs, dec_mask_tokens)

sent_loss_kl = compute_kl_loss(means_infer, logvars_infer) # KL divergence b/w latent dist & gaussian std

## optimizer

In [17]:
beta = tf.Variable(config.beta, name='beta', trainable=False) if config.warmup > 0 else tf.constant(1., name='beta')
update_beta = tf.assign_add(beta, 1./(config.warmup*len(train_batches)))
loss = sent_loss_recon + beta * sent_loss_kl

# define optimizer
if config.opt == 'Adam':
    optimizer = tf.train.AdamOptimizer(config.lr)
elif config.opt == 'Adagrad':
    optimizer = tf.train.AdagradOptimizer(config.lr)
    
grad_vars = optimizer.compute_gradients(loss)
clipped_grad_vars = [(tf.clip_by_value(grad, -config.grad_clip, config.grad_clip), var) for grad, var in grad_vars]

opt = optimizer.apply_gradients(clipped_grad_vars)

# run model 

In [18]:
def idxs_to_sents(token_idxs, config, idx_to_word):
    sents = []
    for sent_idxs in token_idxs:
        tokens = []
        for idx in sent_idxs:
            if idx == config.EOS_IDX: break
            tokens.append(idx_to_word[idx])
        sent = ' '.join(tokens)
        sents.append(sent)
    return sents

In [19]:
def get_loss(sess, batches):
    losses = []
    for ct, batch in batches:
        feed_dict = get_feed_dict(batch, mode='test')
        loss_batch = sess.run(loss, feed_dict = feed_dict)
        losses.append(loss_batch)
    loss_mean = np.mean(losses)
    return loss_mean

In [20]:
def print_sample(sample_batch):
    feed_dict = get_feed_dict(sample_batch)
    pred_token_idxs = sess.run(output_token_idxs, feed_dict = feed_dict)
    true_token_idxs = [sent_idxs for instance in sample_batch for sent_idxs in instance.token_idxs]
    
    assert len(pred_token_idxs) == len(true_token_idxs)
    
    pred_sents = idxs_to_sents(pred_token_idxs, config, idx_to_word)
    true_sents = idxs_to_sents(true_token_idxs, config, idx_to_word)
    
    for true_sent, pred_sent in zip(true_sents, pred_sents):        
        print('True: %s' % true_sent)
        print('Pred: %s' % pred_sent)
        
def print_beam_sample(sample_batch):
    feed_dict = get_feed_dict(sample_batch, mode='test')
    pred_token_idxs = sess.run(beam_output_token_idxs, feed_dict = feed_dict)
    true_token_idxs = [sent_idxs for instance in sample_batch for sent_idxs in instance.token_idxs]
    
    assert len(pred_token_idxs) == len(true_token_idxs)
    
    pred_sents = idxs_to_sents(pred_token_idxs, config, idx_to_word)
    true_sents = idxs_to_sents(true_token_idxs, config, idx_to_word)
    
    for true_sent, pred_sent in zip(true_sents, pred_sents):        
        print('True: %s' % true_sent)
        print('Pred: %s' % pred_sent)
        
def interpolate(sent_1, sent_2):
    sent_tokens_1 = word_tokenize(sent_1)
    sent_tokens_2 = word_tokenize(sent_2)
    
    sent_idxs_1 = [word_to_idx[token] if token in word_to_idx else config.UNK_IDX for token in sent_tokens_1]
    sent_idxs_2 = [word_to_idx[token] if token in word_to_idx else config.UNK_IDX for token in sent_tokens_2]
    feed_input_token_idxs_list = [sent_idxs_1, sent_idxs_2]
    
    feed_input_token_idxs = pad_sequences(feed_input_token_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    sent_l = np.array([len(sent_idxs) for sent_idxs in feed_input_token_idxs_list], np.int32)
    
    _latents_input = sess.run(latents_input,
                              feed_dict={t_variables['input_token_idxs']: feed_input_token_idxs, 
                                         t_variables['sent_l']: sent_l,  
                                         }
                             )
    
    sess.run()
    return _latents_input

In [25]:
if 'sess' in globals(): sess.close()
sess = tf.Session()

sess.run(tf.global_variables_initializer())

logs = []
losses_train = []
ppls_train = []
loss_min = np.inf
beta_eval = 1.
epoch = 0
saver = tf.train.Saver(max_to_keep=10)

In [ ]:
if len(logs) == 0:
    cmd_rm = 'rm -r %s' % config.modeldir
    res = subprocess.call(cmd_rm.split())

    cmd_mk = 'mkdir %s' % config.modeldir
    res = subprocess.call(cmd_mk.split())

time_start = time.time()
for epoch in range(config.epochs):
    for ct, batch in train_batches:
        feed_dict = get_feed_dict(batch)
        if config.warmup > 0 and beta_eval < 1.0: sess.run(update_beta)

        _, loss_batch, sent_loss_recon_batch, sent_loss_kl_batch = \
        sess.run([opt, loss, sent_loss_recon, sent_loss_kl], feed_dict = feed_dict)

        losses_train += [[loss_batch, sent_loss_recon_batch, sent_loss_kl_batch]]

        if ct%config.log_period==0:
            time_dev = time.time()
            loss_train, sent_loss_recon_train, sent_loss_kl_train = np.mean(losses_train, 0)
            loss_dev = get_loss(sess, dev_batches)
            
            if loss_dev < loss_min:
                loss_min = loss_dev
                saver.save(sess, config.modelpath, global_step=epoch*10000+ct)

            if config.warmup > 0: beta_eval = beta.eval(session=sess)

            clear_output()
            time_finish = time.time()
            time_log = int(time_finish - time_start)
            time_log_dev = int(time_finish - time_dev)
            logs += [(time_log, time_log_dev, epoch, ct, loss_train, sent_loss_recon_train, sent_loss_kl_train, loss_dev, beta_eval)]
            for log in logs:
                print('%03d[s], %02d[s], Ep: %02d, Ct: %05d|TR LOSS: %.2f LM NLL: %.2f, KL: %.2f|DE LOSS: %.2f|BETA: %.6f' %  log)

            print_sample(batch)

            time_start = time.time()

027[s], 26[s], Ep: 00, Ct: 00000|TR LOSS: 10.34 LM NLL: 10.34, KL: 0.00|DE LOSS: 10.34|BETA: 0.001000
093[s], 26[s], Ep: 00, Ct: 00500|TR LOSS: 7.39 LM NLL: 7.37, KL: 23.52|DE LOSS: 6.85|BETA: 0.001035
True: delaware state police have arrested a philadelphia man they say discarded a weapon while they chased him following a domestic dispute
Pred: the a a a a a a a
True: it happened about noon thursday at the wal mart parking lot in dover
Pred: the the a # a # a
True: police say a man and his girlfriend got into an argument as they drove in the parking lot
Pred: the the a a a a # a
True: troopers say the man hit the woman , dragged her out of the car , pinned her to the ground and took her cell phone
Pred: the a a a # a a a # a to # a
True: authorities say troopers spotted the man running and saw him put a handgun under a car in a parking lot
Pred: the the a the the #
True: he was caught and arrested
Pred: the the the the
True: twenty four year old keith denmark is charged with robbery ,

In [121]:
print_beam_sample(batch)

FailedPreconditionError: Attempting to use uninitialized value dense_5/kernel
	 [[node dense_5/kernel/read (defined at <ipython-input-116-329bc6abf768>:4)  = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_5/kernel)]]
	 [[{{node decoder_7/while/Switch_1/_1105}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_564_decoder_7/while/Switch_1", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](^_cloopdecoder_7/while/BeamSearchDecoderStep/tensor_gather_helper/GatherV2/axis/_1011)]]

Caused by op 'dense_5/kernel/read', defined at:
  File "/home/m-isonuma/.pyenv/versions/anaconda2-5.3.0/envs/py36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/m-isonuma/.pyenv/versions/anaconda2-5.3.0/envs/py36/lib/python3.6/runpy.py", line 85, in _run_

In [102]:
sent_1 = 'It is a dog'
sent_2 = 'It is a president'
interpolate(sent_1, sent_2)

array([[-1.2384069 , -0.3807991 , -1.1647905 ,  0.7314516 , -0.20042299,
         0.8870377 , -1.4098704 , -1.1511734 ,  0.38629064,  0.98834074,
        -0.8180504 , -0.15959802,  0.06843489, -0.46147212, -1.8696944 ,
        -0.5360981 , -0.5004562 , -0.7060725 ,  1.5187515 , -0.17914969,
        -0.32096705, -0.25406975,  0.43872547,  0.20233467, -0.0175207 ,
         1.4343544 , -0.35730684, -1.354493  ,  0.5174452 ,  2.9534483 ,
        -1.5597805 , -0.52366066],
       [-0.78195465,  0.0574708 , -1.331151  ,  0.35355008,  0.03828494,
         0.5575796 , -1.0602851 ,  0.11334677, -0.37056714, -0.29048857,
        -1.0173718 , -0.535958  ,  0.56922436, -1.588103  , -2.573833  ,
         0.36844906, -0.81400114, -0.6146785 ,  1.244829  ,  0.62736154,
        -1.0616951 ,  0.40555957, -0.09158325, -1.2912164 ,  0.22092906,
         1.9417524 , -1.7419215 , -1.4243282 ,  0.09032765,  3.0320437 ,
         0.3378294 , -1.2621846 ]], dtype=float32)

In [99]:
word_tokenize(sent_1)

['It', 'is', 'a', 'dog']

# confirm variables

In [110]:
_logvars, _means, _kl_losses, _latents, _output_logits = sess.run([logvars, means, kl_losses, latents, output_logits], feed_dict=feed_dict)


In [111]:
_logvars.shape, _means.shape, _kl_losses.shape, _latents.shape

((32, 32), (32, 32), (32,), (32, 32))

In [112]:
_output_logits

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)

In [109]:
_output_logits, _dec_target_idxs_do, _dec_mask_tokens_do, _recon_loss, _kl_losses, _ = sess.run([output_logits, dec_target_idxs_do, dec_mask_tokens_do, recon_loss, kl_losses, opt], feed_dict=feed_dict)


NameError: name 'dec_target_idxs_do' is not defined

In [44]:
tf.reduce_max(output_logits, 2).eval(session=sess, feed_dict=feed_dict).shape

(120, 46)

In [31]:
_output_logits.shape, _dec_target_idxs_do.shape, _dec_mask_tokens_do.shape

((120, 46, 20000), (120, 46), (120, 46))

In [32]:
_logits = np.exp(_output_logits) / np.sum(np.exp(_output_logits), 2)[:, :, None]

In [33]:
_idxs = _dec_target_idxs_do

In [35]:
_losses = np.array([[-np.log(_logits[i, j, _idxs[i, j]]) for j in range(_idxs.shape[1])] for i in range(_idxs.shape[0])]) * _dec_mask_tokens_do

In [36]:
np.sum(_losses)/np.sum(_dec_mask_tokens_do)

9.903732

In [37]:
_recon_loss

9.903732

In [38]:
_kl_losses.shape

(120,)